Part 1: Needleman-Wunsch Scoring

In [17]:
# Gap penalty
a_gap = 1


# Alignment/misalignment penalty
def a(x, y):
    if x == y:
        return 0
    return 1

#needleman wunsch alignment
def needleman_wunch(X, Y):
    """Computes minimum alignment score for input strings X and Y"""
    # Shortcults to the input string lengths for readability.
    m = len(X)
    n = len(Y)
    # Initialize the penalty table P with m rows and n columns
    P = [[0 for _ in range(n + 1)] for _ in range(m + 1)]
    # Initialize the base cases for P(0,0), P(i,0), and P(0,j)
    for i in range(1, m + 1):
        P[i][0] = i * a_gap
    for j in range(1, n + 1):
        P[0][j] = j * a_gap
    # Fill the rest of the table. Remember to offset string slices 
    # by 1 to account for the fact that the first character of 
    # a string is at index 0, while the first row/column of P
    # corresponds to the empty string.
    for i in range(1, m + 1):
        for j in range(1, n + 1):
            P[i][j] = min(
                P[i - 1][j - 1] + a(X[i-1], Y[j-1]),
                P[i - 1][j] + a_gap,
                P[i][j - 1] + a_gap,
            )
    # Done
    return P

#print the penalty table
def fancy_print(P):
    """A quick method to print the penalty table nicely."""
    for i in range(len(P)):
        for j in range(len(P[0])):
            print(f"{P[i][j]:5d} ", end="")
        print()

# Quick test:
X = "BICYCLE"
Y = "CYCLE"
P = needleman_wunch(X, Y)
fancy_print(P)

    0     1     2     3     4     5 
    1     1     2     3     4     5 
    2     2     2     3     4     5 
    3     2     3     2     3     4 
    4     3     2     3     3     4 
    5     4     3     2     3     4 
    6     5     4     3     2     3 
    7     6     5     4     3     2 


Now, we implement a trace-back in order to align two strings and calculate the number of matches, mismatches, and gaps. 

In [18]:
def find_alignment(P_matrix, word1, word2): 
    
    #find out what index to start at for each word
    i = len(word1) 
    j = len(word2)
    
    #start lists for each word's alignment 
    word1_align = []
    word2_align = []
    
    #start counts for mismatches, gaps, and matches
    match_count = 0
    mismatch_count = 0
    gap_count = 0
    
    while j != 0 and i != 0: #while we haven't reached the top left corner of our matrix
        
        #name a current coordinate to start with, which will be the bottom right at first
        current = P_matrix[i][j]
        
        #what are the match and gap scores?
        match = P_matrix[i - 1][j -1]
        gap_above = P_matrix[i - 1][j]
        gap_left = P_matrix[i][j - 1]
        
        if match == current and word1[i - 1] == word2[j - 1]: #if the match score is equal to current score, that means this is a match
            word1_align.append(word1[i - 1])
            word2_align.append(word2[j - 1])
            
            i -= 1
            j -= 1
            
            match_count += 1
            
        else: #this is not a match 
            
            #we look for the minimum of these scores, which is probably the one we took from: 
            min_score = min(match, gap_above, gap_left)
            
            #now we trace back to the coordinate we took from
            if gap_above == min_score: 
                
                #we have a gap
                word1_align.append(word1[i - 1])
                word2_align.append("-")
                
                i -= 1
                j = j
                
                gap_count += 1
                
            elif gap_left == min_score: 
                
                #we have a gap, but it's in word 1 instead of word 2
                word1_align.append("-")
                word2_align.append(word2[j - 1])
                
                i = i
                j -= 1
                
                gap_count += 1
            
            elif match == min_score: 
                
                #we have a mismatch
                word1_align.append(word1[i - 1])
                word2_align.append(word2[j - 1])
                
                i -= 1
                j -= 1
                
                mismatch_count += 1       
    
    #once one of them equals 0, we check from there to see which word still has letters left  
    if i == 0 and j != 0: 
        #word1_align.append(word1[0])
        word1_align.append("-" * j)
        
        #word2_align.append(word2[0:j + 1])
        word2_align.append(word2[0:j])
        
        gap_count += j
    
    elif i != 0 and j == 0: 
        #word2_align.append(word2[0])
        word2_align.append("-" * i)
        
        #word1_align.append(word1[0:i + 1])
        word1_align.append(word1[0:i])
        
        gap_count += i
        
    word1_align.reverse()
    word2_align.reverse()
    
    #ok, and now we return everything important that we collected
    return(word1_align, word2_align, match_count, mismatch_count, gap_count)
    

In [19]:
if __name__ == "__main__":
    tests = [
        ["CRANE", "RAIN"],
        ["CYCLE", "BICYCLE"],
        ["ASTRONOMY", "GASTRONOMY"],
        ["INTENTION", "EXECUTION"],
        ["AGGTAB", "GXTXAYB"],
        ["GATTACA", "GCATGCU"],
        ["DELICIOUS", "RELIGIOUS"],
]

    for word1, word2 in tests:
        #perform needleman wunsch to create matrix: 
        P_matrix = needleman_wunch(word1, word2)
        
        word1_align, word2_align, match_count, mismatch_count, gap_count = find_alignment(P_matrix, word1, word2)
        
        word1_align = "".join(word1_align)
        word2_align = "".join(word2_align)
        
        len_longer = max(len(word1), len(word2))
        
        word1 = word1 + (" " * (len_longer - len(word1)))
        word2 = word2 + (" " * (len_longer - len(word2)))
        
        #we want to do the traceback, so show the alignment pattern
        #we also want to include the number of matches, mismatches, and gaps
        print(f'{word1} -->  |{word1_align}|    matches: {match_count}, mismatches: {mismatch_count}')
        print(f'{word2} -->  |{word2_align}|    gaps: {gap_count}')
        print()


CRANE -->  |CRA-NE|    matches: 3, mismatches: 0
RAIN  -->  |-RAIN-|    gaps: 3

CYCLE   -->  |--CYCLE|    matches: 5, mismatches: 0
BICYCLE -->  |BICYCLE|    gaps: 2

ASTRONOMY  -->  |-ASTRONOMY|    matches: 9, mismatches: 0
GASTRONOMY -->  |GASTRONOMY|    gaps: 1

INTENTION -->  |INTEN-TION|    matches: 5, mismatches: 3
EXECUTION -->  |EX-ECUTION|    gaps: 2

AGGTAB  -->  |AGGT-A-B|    matches: 4, mismatches: 1
GXTXAYB -->  |-GXTXAYB|    gaps: 3

GATTACA -->  |G-ATTACA|    matches: 4, mismatches: 2
GCATGCU -->  |GCATG-CU|    gaps: 2

DELICIOUS -->  |DELICIOUS|    matches: 7, mismatches: 2
RELIGIOUS -->  |RELIGIOUS|    gaps: 0



Some of these alignments are slightly different from the example alignments in the key, but they all have the same number of mismatches, matches, and gaps. My impression of this is that some sequences might technically have multiple optimal alignments, and my code probably checks for a gap before it checks for a mismatch because of the order of if/else statements in my find_alignment() function. I would guess this is why the alignments are slightly different, but I don't think that either is incorrect. If I wanted to be the same as the key, I would probably check for mismatch before gap in my find_alignment() statement. 